<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/1_4_MODELLING_DATES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# our own package for reading the data
!pip install sddk
import sddk

import pandas as pd

In [1]:
# (1) import requests & scipy (or: first install them with pip)
import requests
import scipy
# (2) path to the script
url = "https://raw.githubusercontent.com/sdam-au/modelling_temporal_uncertainty/master/scripts/modelling_distributions.py"
# (3) make a request to this path
resp = requests.get(url)
# (4) execute the script
exec(resp.content)

In [ ]:
# login to sciencedata 
conf = sddk.configure("SDAM_root", "648597@au.dk")